<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Predicting Medical Expenses in Healthcare 
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
In the dynamic landscape of healthcare and insurance, understanding the factors influencing medical expenses is paramount. Accurate predictions of medical expenses enable healthcare providers and insurance companies to better manage costs, improve resource allocation, and enhance customer satisfaction. By leveraging regression models, we can develop robust predictive analytics that help in understanding and forecasting medical expenses based on various factors such as age, BMI, smoking status, and more.<br>Healthcare providers and insurance companies face challenges in accurately predicting medical expenses for individuals. Unanticipated high medical costs can strain resources, lead to financial losses, and affect the overall quality of service. The objective is to build a regression model that accurately predicts individual medical expenses, allowing for better financial planning and risk management.<br>
    In this demo we have taken a dataset which has information about age, sex, BMI, smoking status, number of children and region which serves as a comprehensive foundation for predictive modeling in healthcare costs. By utilizing this dataset, we can develop accurate machine learning models to forecast medical expenses for new policyholders. Each variable provides critical insights: age correlates with higher medical expenses due to chronic conditions, BMI indicates obesity-related health risks, smoking status highlights costs associated with smoking-related diseases, and the number of children reflects the medical needs of larger families. Regional differences further illustrate variations in healthcare costs and accessibility. Training models on this dataset allows insurance companies to identify key cost drivers, predict future expenses with precision, and tailor pricing strategies, thereby improving decision-making, fairness, and efficiency in insurance policy pricing.
</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Business Values</b></p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
            <li> Accurate Prediction of Medical Expenses: Develop a regression model to predict medical expenses based on key factors.</li>
            <li>Identify Key Cost Drivers: Determine the most significant factors influencing medical expenses. </li>
            <li>Enhance Resource Allocation: Enable healthcare providers and insurers to allocate resources more effectively. </li>
            <li>Improve Customer Satisfaction: Offer better pricing strategies and personalized plans to customers.</li>
 </ul>   
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>  
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>One of the primary challenges faced by AI/ML projects is data collection and standardization. Data pre-processing alone can consume 70 to 80% of the time and resources before model creation can begin. With Vantage's Clearscape in-DB functions, these standardization and pre-processing steps can be performed at scale, significantly reducing the time and data movement required. Additionally, leveraging the Teradata Python Package, particularly the teradataml OpenSourceML component, allows users to utilize popular open-source machine learning packages like scikit-learn directly within the database environment. This integration eliminates the need to transfer data to the client for analysis, thereby streamlining the workflow and enhancing efficiency. The OpenSourceML package simplifies the incorporation of open-source machine learning functionalities into Vantage, providing a consistent interface for executing these algorithms. Users can employ familiar syntax and arguments, facilitating a smooth transition from traditional open-source environments to the Vantage platform.
</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1. Connect to Vantage</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's start by importing the libraries needed.</p>

In [ ]:
# Standard libraries
import getpass
import warnings

# Third-party libraries
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns

# Teradata libraries
from teradataml import *
display.max_rows = 5

# Suppress warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql("SET query_band='DEMO=PP_Health_Insurance_Costs_Python.ipynb;' UPDATE FOR SESSION;")

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_Health_cloud');"
 # takes about 30 seconds, estimated space: 0 MB
#%run -i ../run_procedure.py "call get_data('DEMO_Health_local');" 
# takes about 30 seconds, estimated space: 1 MB

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Initial Data Set</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
Let us start by analyzing the data.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_Health","Health_Insurance_Costs"))
tdf.head()

In [ ]:
tdf.shape

In [ ]:
tdf.tdtypes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
From the above statements we can see the columns, the size of dataset and the columntypes.<br> Now let us apply some Data Exploration Functions avaiable in Clearscape Analytics. First let us start by ColumnSummary.<br><b>ColumnSummary </b>function  displays Column name, datatype and other demographics like count of NULLs etc for each specified input table column</p>

In [ ]:
colsum = ColumnSummary(data=tdf,
                        target_columns=[':']
                       )
colsum.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>CategoricalSummary </b>function displays the distinct values and their counts for each specified input table column.</p>

In [ ]:
catsum = CategoricalSummary(data=tdf,
                             target_columns=['Sex','Smoker','Region']
                            )
 
catsum.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In our data we have Smoker as character with values yes and no, first let us convert this to integer datatype by converting yes to 1 and no to 0.</p>

In [ ]:
#converting smoker to integer
tdf = tdf.assign(Smoker = case([(tdf.Smoker == 'yes', 1)], else_ = 0 ))
tdf

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next we gather statistics using <b>UnivariateStatistics</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Univariate analysis is the simplest form of analyzing data. <b>UnivariateStatistics</b> displays descriptive statistics for each specified numeric input table column.
</p>

In [ ]:
#stats analysis
obj = UnivariateStatistics(newdata=tdf,
                               target_columns=['Age','BMI','Smoker','Children','Charges'],
                               stats=['MEAN','MEDIAN','MODE','KURTOSIS','SKEWNESS','STANDARD DEVIATION',
                                      'SUM','PERCENTILES','MINIMUM','MAXIMUM']
                          )

    # Print the result DataFrame.
obj.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let us pivot the result of UnivariateStatistics for better readability.</p>

In [ ]:
#pivot the result for more readbility
p = obj.result
p2=p.pivot(columns=p.ATTRIBUTE, aggfuncs=p.StatValue.max())
p2=p2.assign(drop_columns=True,
             StatName=p2.StatName,
             Smoker=p2.max_statvalue_smoker,
             Charges=p2.max_statvalue_charges,
             BMI=p2.max_statvalue_bmi,
             Age=p2.max_statvalue_age,
             Children=p2.max_statvalue_children)
p2.head(20)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>From above we can see the Statictics generated for the numerical columns in the dataset.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Exploratory Data Analysis </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> Let us plot some graphs of the data to see if we can identify some patterns in the data.<br> Teradata machine learning package (tdml) has some basic functionality to plot graphs, by using this we ensure that the data is not moved outside and large plots can be created with ease.<br> For more complex graphs we can integrate with available pyhton packages, data movement will be there but we can ensure it can be done for specific scenarios. </p>

In [ ]:
#creating separate dataframe for plotting
df=tdf

In [ ]:
df.plot(x=df.BMI, 
        y=df.Charges, 
        kind="scatter",
        color="blue", 
        grid_color='grey',
        xlabel='BMI', 
        ylabel='Charges',
        grid_linestyle="-",
        grid_linewidth= 0.5, 
        marker="o",
        markersize=7,
        title="BMI vs Charges")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> In the plot above we can see how the Charges are changing based on the BMI.</p>

In [ ]:
#converting female =1 and male = 0 for plotting
df = df.assign(sex_int = case([(df.Sex == 'female', 1)], else_ = 0 ))
df.plot(x=df.sex_int, 
        y=df.Charges, 
        kind="scatter",
        color="blue", 
        grid_color='grey',
        xlabel='Sex', 
        ylabel='Charges',
        grid_linestyle="-",
        grid_linewidth= 0.5, 
        marker="o",
        markersize=7,
        title="Sex(1-female,0-male) vs Charges")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> In the plot above we can see how the charges are changing based on the gender of the patient.</p>

In [ ]:
df.plot(x=df.Age, 
        y=df.Charges, 
        kind="scatter",
        color="blue", 
        grid_color='grey',
        xlabel='Age', 
        ylabel='Charges',
        grid_linestyle="-",
        grid_linewidth= 0.5, 
        marker="o",
        markersize=7,
        title="Age vs Charges")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> In the plot above we can see the charges based on the age of the patient.</p>

In [ ]:
df.plot(x=df.Smoker, 
        y=df.Charges, 
        kind="scatter",
        color="blue", 
        grid_color='grey',
        xlabel='Smoker', 
        ylabel='Charges',
        grid_linestyle="-",
        grid_linewidth= 0.5, 
        marker="o",
        markersize=7,
        title="Smoker(1-yes, 0-no) vs Charges")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> In the plot above we can see how smoking is affecting the charges.</p>

In [ ]:
df = df.assign(region_int = case([(df.Region == 'southeast', 1),
                                  (df.Region == 'northwest', 2),
                                  (df.Region == 'northeast', 3)
                                 ], else_ = 0 ))
df.plot(x=df.region_int, 
        y=df.Charges, 
        kind="scatter",
        color="blue", 
        grid_color='grey',
        xlabel='Region', 
        ylabel='Charges',
        grid_linestyle="-",
        grid_linewidth= 0.5, 
        marker="o",
        markersize=7,
        title="Region(0-southwest,1-southeast,2-northwest,3-northeast) vs Charges")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> In the plot above we can see the changes in charges based on the region of the patient.</p>

In [ ]:
df.plot(x=df.Children, 
        y=df.Charges, 
        kind="scatter",
        color="blue", 
        grid_color='grey',
        xlabel='Region', 
        ylabel='Charges',
        grid_linestyle="-",
        grid_linewidth= 0.5, 
        marker="o",
        markersize=7,
        title="Children vs Charges")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> In the plot above we can see the change in charges based on the number of children.</p>

In [ ]:
df1=df.to_pandas().reset_index()
boxplot = df1.boxplot(column=['BMI'])

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The above plot helps us to check the outlier values in the BMI.</p>

In [ ]:
plt.figure(figsize=(30,28))
for i, col in enumerate( ['Age','BMI', 'Children','Charges']):
    plt.subplot(3, 3, i+1)
    sns.histplot(data = df1,
            x = col,
            kde = True,
            bins = 30,
            color = 'green')

plt.show()

In [ ]:
plt.figure(figsize=(12,9))
for i,col in enumerate(['Sex','Smoker','Region', 'Children']):
    plt.subplot(3,2,i+1)
    x=df1[col].value_counts().reset_index()
    plt.title(col)
    plt.pie(x=x['count'],labels=x[col],autopct="%0.1f%%",colors=sns.color_palette('muted'))

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The above plot shows the percentage of each column in the dataset we have.</p>

In [ ]:
d= df1.drop(columns=['Sex', 'Region'])
data_corr = d.corr()
sns.heatmap(data=data_corr,annot=True)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The above plot shows the correlation of each attribute with each other.</p>

In [ ]:
sns.pairplot(df1, vars = df1.iloc[:, [1] + list(range(3, 6))+[7]])
plt.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The above plots show the pairplot of attributes with each other.</p>

In [ ]:
sns.scatterplot(data=df1,x=df1.Charges,y=df1.Smoker,hue=df1.Age)
plt.show()

In [ ]:
sns.barplot(data=df1,x=df1.Smoker,y=df1.Charges,estimator=np.mean)
plt.show()

In [ ]:
sns.barplot(data=df1,x=df1.Region,y=df1.Charges,estimator=np.mean)
plt.show()

In [ ]:
df.plot(y=df.Age, 
        x=df.Charges, 
        kind="scatter",
        color="blue", 
        grid_color='grey',
        ylabel='Age', 
        xlabel='Charges',
        grid_linestyle="-",
        grid_linewidth= 0.5, 
        marker="o",
        markersize=7,
        title="Age vs Charges")

<p style = 'font-size:20px;font-family:Arial;color:#00233C'> <b> 4. Feature Engineering Functions  </b> </p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>OneHotEncodingFit </b>outputs a table of attributes and categorical values to input to <b>OneHotEncodingTransform </b> which encodes them as one-hot numeric vectors.</p>

In [ ]:
tdf

In [ ]:
#one hot encoding for sex & region
#scaling for BMI
hot_fit = OneHotEncodingFit(data=tdf,
                                is_input_dense=True,
                                target_column=['Sex','Region'],
                                category_counts=[2,4],
                                approach="auto")
 
# Print the result DataFrame.
hot_fit.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>ScaleFit and ScaleTransform </b>scales specified input
table columns i.e perform the specific scale methods like standard deviation, mean etc to the input columns </p>

In [ ]:
scale_fit = ScaleFit(data=tdf,
                       target_columns="BMI",
                       scale_method="RANGE",
                       miss_value="KEEP",
                       global_scale=False)
scale_fit.output

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b> ColumnTransformer  </b> </p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ColumnTransformer function transforms the entire dataset in a single operation. You only need
to provide the FIT tables to the function, and the function runs all transformations that you require in a
single operation. Running all the it table transformations together in one-go gives approx. 30% performance improvement over running each transformation sequentially.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let us put all the fit tables we have created and transform the dataset</p>

In [ ]:
out1 = ColumnTransformer(input_data=tdf,
                         scale_fit_data=scale_fit.output,
                         onehotencoding_fit_data=hot_fit.result,
                        )

In [ ]:
out1.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can drop the extra columns and rename the columsn to create dataframe that can be used for model creation.</p>

In [ ]:
t=out1.result
transformed_df = out1.result.assign(drop_columns=True
                  ,rID=t.rID  
                  ,Age=t.Age 
                  ,BMI=t.BMI
                  ,Children=t.Children
                  ,Smoker=t.Smoker
                  ,Charges=t.Charges
                  ,Sex_female=t.Sex_0      
                  ,Sex_male=t.Sex_1     
                  ,Region_SW=t.Region_3     
                  ,Region_SE=t.Region_2        
                  ,Region_NW=t.Region_1    
                  ,Region_NE=t.Region_0             
              ) 

transformed_df

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>5. Model Creation</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Train and Test Datasets</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now let us divide our data in training and test datasets for model creation. We can do this by using Clearscape Analytics TrainTestSplit function.</p>

In [ ]:
TrainTestSplit_out = TrainTestSplit(
                                    data = transformed_df,
                                    id_column = "rID",
                                    train_size = 0.75,
                                    test_size = 0.25,
                                    seed = 25
                                   )

In [ ]:
df_train = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
df_test = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>teradataml Open-Source Machine Learning Functions</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Teradata Package for Python introduces teradataml open-source machine learning functions(teradataml OpenSourceML) which exposes most of the functionality of open-source packages like scikit-learn. With teradataml open-source machine learning functions, we can run these open-source packages without needing to pull the data to client.It offers a simple interface object for the open-source packages, allowing them to be used with the same syntax and arguments as the actual open-source packages' functions and classes.<br>Let us use the use the scikit learn models for our demo but first lets divide our dataset to train and test datasets.</p> 

In [ ]:
X_train = df_train.drop(['Charges','rID'], axis = 1)
y_train = df_train.select(["Charges"])
X_test = df_test.drop(['Charges','rID'], axis = 1)
y_test = df_test.select(["Charges"])

In [ ]:
X_train

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1 Linear Regression</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Linear regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables. It assumes a linear relationship between the variables and aims to find the best-fitting straight line that minimizes the sum of squared differences between the observed values and the predicted values. The resulting equation can be used to predict the value of the dependent variable based on the values of the independent variables.<br> We are using the LinearRegression classifier available in the teradataml OpenSourceML library which simulates the same behaviour as scikit-learn LinearRegression classifier.</p>

In [ ]:
from teradataml import td_sklearn as osml
lr = osml.LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
lr.score(X_test,y_test)

In [ ]:
lr.predict(X_test)

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2 RandomForestRegressor</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The RandomForestRegressor is an ensemble learning method used for regression tasks that combines the predictions of multiple decision trees to improve accuracy and control overfitting. It operates by constructing a multitude of decision trees during training, where each tree is built using a random subset of the data and a random subset of features, which enhances diversity among the trees. When making predictions, the RandomForestRegressor aggregates the outputs of all individual trees, typically by averaging their predictions, to produce a final output. This approach not only increases predictive performance but also provides insights into feature importance, making it a powerful tool for handling complex datasets with non-linear relationships and interactions among variables.</p>

In [ ]:
model_rfr = osml.RandomForestRegressor()
model_rfr.fit(X_train,y_train)

In [ ]:
model_rfr.score(X_test,y_test)

In [ ]:
model_rfr.predict(X_test)

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this notebook we have seen that by implementing  regression model to predict medical expenses, healthcare providers and insurance companies can significantly enhance their financial planning and risk management capabilities. This approach not only helps in understanding the key drivers of medical costs but also leads to improved customer satisfaction through personalized and cost-effective healthcare solutions. We have also seen the seamless integration of Teradata Vantage and ClearScape Analytics inDb functions with OpensourceML functions, empowering users with scalable data processing capabilities. By leveraging these functionalities within the database environment, users gain the flexibility to analyze vast amounts of data without the need to transfer it to client tools. This integration not only streamlines the workflow but also provides users with the choice of utilizing open-source machine learning functions directly within the database, further enhancing efficiency and expanding the range of analytical tools available.</p>

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>5. Cleanup</b></p>


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Database and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_Health');" 
#Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Required Materials</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let’s look at the elements we have available for reference for this notebook:</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Filters: </b></p>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Industry:</b> Healthcare</li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Functionality:</b> Open-Source Machine Learning Functions</li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Use Case:</b> Predicting Healthcare medical costs</li></p>
    <p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Related Resources:</b></p>
<li><a href='https://usc-word-edit.officeapps.live.com/we/%E2%80%A2%09https:/www.teradata.com/Blogs/Predicting-Heart-Failure-with-Teradata'>Saving Lives, Saving Costs: Predicting Heart Failure with Teradata</a></li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href='https://www.teradata.com/customers/texas-health-resources'>Texas Health:Transforming health care for 7 million patients through the power of data</a></li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href='https://www.teradata.com/customers/siemens-healthineers'>Siemens Healthineers</a></li>


<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            © 2024 Teradata. All rights reserved.
        </div>
    </div>
</footer>